In [1]:
# Check P2M, M2P and M2M
import numpy as np
from pcm_fmm import pcm_fmm
p = 10
ngrid = 6
vscales, w, grid, vgrid = pcm_fmm.init_globals(p, p, ngrid)
vscalesm = vscales[:(p+1)*(p+1)]
src = np.array([0.5,0.4,0.3], dtype=np.float64)
dst = np.array([1, 1.1, 2.2], dtype=np.float64)
p2p = np.linalg.norm(src-dst)**-1
sph1_c = np.zeros(3)
sph1_r = 1
sph1_coef = pcm_fmm.fmm_p2m(src-sph1_c, sph1_r, p, vscalesm)
m2p = pcm_fmm.fmm_m2p(dst-sph1_c, sph1_r, p, vscalesm, sph1_coef)
print("|| P2M + M2P - P2P ||  /  || P2P || =", abs((m2p-p2p)/p2p))
sph2_c = -np.ones(3)
sph2_r = sph1_r + np.linalg.norm(sph2_c-sph1_c)
sph2_coef = pcm_fmm.fmm_p2m(src-sph2_c, sph2_r, p, vscalesm)
sph2_coef_m2m = np.zeros_like(sph2_coef)
pcm_fmm.fmm_m2m_baseline(sph1_c-sph2_c, sph1_r, sph2_r, p, vscalesm, sph1_coef, sph2_coef_m2m)
print("|| P2M(1) + M(1)2M(2) - P2M(2) ||  /  || P2M(2) || =",
      np.linalg.norm(sph2_coef_m2m-sph2_coef) / np.linalg.norm(sph2_coef))
sph3_c = np.array([0, 0, -1])
sph3_r = sph1_r + np.linalg.norm(sph3_c-sph1_c)
sph3_coef = pcm_fmm.fmm_p2m(src-sph3_c, sph3_r, p, vscalesm)
sph3_coef_m2m = np.zeros_like(sph3_coef)
pcm_fmm.fmm_m2m_ztranslate((sph1_c-sph3_c)[2], sph1_r, sph3_r, p, vscalesm, sph1_coef, sph3_coef_m2m)
print("|| P2M(1) + M(1)TZ2M(3) - P2M(3) ||  /  || P2M(3) || =",
      np.linalg.norm(sph3_coef_m2m-sph3_coef) / np.linalg.norm(sph3_coef))
sph4_c = sph1_c
sph4_r = 3*sph1_r
sph4_coef = pcm_fmm.fmm_p2m(src-sph4_c, sph4_r, p, vscalesm)
sph4_coef_m2m = np.zeros_like(sph4_coef)
pcm_fmm.fmm_m2m_baseline(sph1_c-sph4_c, sph1_r, sph4_r, p, vscalesm, sph1_coef, sph4_coef_m2m)
print("|| P2M(1) + M(1)2M(4) - P2M(4) ||  /  || P2M(4) || =",
      np.linalg.norm(sph4_coef_m2m-sph4_coef) / np.linalg.norm(sph4_coef))

|| P2M + M2P - P2P ||  /  || P2P || = 1.2811025058542572e-07
|| P2M(1) + M(1)2M(2) - P2M(2) ||  /  || P2M(2) || = 3.244277800823038e-16
|| P2M(1) + M(1)TZ2M(3) - P2M(3) ||  /  || P2M(3) || = 1.7620736005053075e-16
|| P2M(1) + M(1)2M(4) - P2M(4) ||  /  || P2M(4) || = 7.08553113105978e-17


In [2]:
# Check M2L, L2P and L2L
import numpy as np
from pcm_fmm import pcm_fmm
pl = 10
pm = 10
ngrid = 6
vscales, w, grid, vgrid = pcm_fmm.init_globals(pm, pl, ngrid)
src = np.array([0.5,0.4,0.3], dtype=np.float64)
dst = np.array([1, 1.1, 2.2], dtype=np.float64)
p2p = np.linalg.norm(src-dst)**-1
# Compute multipole coefficients
sph1_c = np.zeros(3)
sph1_r = 2
sph1_coef = pcm_fmm.fmm_p2m(src-sph1_c, sph1_r, pm, vscales[:(pm+1)**2])
# Compute local expansion
sph2_c = np.array([1, 1, 2], dtype=np.float64)
sph2_r = 1.3
sph2_coef = np.zeros((pl+1)*(pl+1))
pcm_fmm.fmm_m2l_baseline(sph1_c-sph2_c, sph1_r, sph2_r, pm, pl, vscales, sph1_coef, sph2_coef)
l2p = pcm_fmm.fmm_l2p(dst-sph2_c, sph2_r, pl, vscales[:(pl+1)**2], sph2_coef)
print("|| P2M + M2L + L2P - P2P ||  /  || P2P || =", abs((l2p-p2p)/p2p))
sph3_c = np.array([1, 1, 2], dtype=np.float64)
sph3_r = 0.6
sph3_coef = np.zeros_like(sph2_coef)
pcm_fmm.fmm_l2l_baseline(sph2_c-sph3_c, sph2_r, sph3_r, pl, vscales[:(pl+1)**2], sph2_coef, sph3_coef)
l2l2p = pcm_fmm.fmm_l2p(dst-sph3_c, sph3_r, pl, vscales[:(pl+1)**2], sph3_coef)
print("|| L(1)2L(2) + L(2)2P - L(1)2P ||  /  || L(1)2P || =", abs((l2l2p-l2p)/l2p))
sph3_c = np.array([1.1, 1.1, 2.1], dtype=np.float64)
sph3_r = 0.6
sph3_coef = np.zeros_like(sph2_coef)
pcm_fmm.fmm_l2l_baseline(sph2_c-sph3_c, sph2_r, sph3_r, pl, vscales[:(pl+1)**2], sph2_coef, sph3_coef)
l2l2p = pcm_fmm.fmm_l2p(dst-sph3_c, sph3_r, pl, vscales[:(pl+1)**2], sph3_coef)
print("|| L(1)2L(3) + L(3)2P - L(1)2P ||  /  || L(1)2P || =", abs((l2l2p-l2p)/l2p))

|| P2M + M2L + L2P - P2P ||  /  || P2P || = 1.2813589395338697e-07
|| L(1)2L(2) + L(2)2P - L(1)2P ||  /  || L(1)2P || = 0.0
|| L(1)2L(3) + L(3)2P - L(1)2P ||  /  || L(1)2P || = 6.946662007989605e-16


In [3]:
# Check rotation-based M2M
import numpy as np
from pcm_fmm import pcm_fmm
p = 20
ngrid = 6
vscales, w, grid, vgrid = pcm_fmm.init_globals(p, p, ngrid)
vscalesm = vscales[:(p+1)*(p+1)]
sph1_c = np.zeros(3)
sph1_r = 1
sph1_coef = np.ones_like(vscalesm)
sph2_c = np.array([0, 0, 10], dtype=np.float64)
sph2_r = sph1_r + np.linalg.norm(sph2_c-sph1_c)
sph3_c = np.array([-1, -2, -10], dtype=np.float64)
sph3_r = sph1_r + np.linalg.norm(sph3_c-sph1_c)
sph2_coef_baseline = np.zeros_like(sph1_coef)
pcm_fmm.fmm_m2m_baseline(sph1_c-sph2_c, sph1_r, sph2_r, p, vscalesm, sph1_coef, sph2_coef_baseline)
sph3_coef_baseline = np.zeros_like(sph1_coef)
pcm_fmm.fmm_m2m_baseline(sph1_c-sph3_c, sph1_r, sph3_r, p, vscalesm, sph1_coef, sph3_coef_baseline)
sph2_coef_reflection = np.zeros_like(sph1_coef)
pcm_fmm.fmm_m2m_reflection(sph1_c-sph2_c, sph1_r, sph2_r, p, vscalesm, sph1_coef, sph2_coef_reflection)
sph3_coef_reflection = np.zeros_like(sph1_coef)
pcm_fmm.fmm_m2m_reflection(sph1_c-sph3_c, sph1_r, sph3_r, p, vscalesm, sph1_coef, sph3_coef_reflection)
tmp2 = sph2_coef_reflection - sph2_coef_baseline
tmp3 = sph3_coef_reflection - sph3_coef_baseline
print("fmm_m2m_reflection:", np.linalg.norm(tmp2) / np.linalg.norm(sph2_coef_baseline),
      np.linalg.norm(tmp3) / np.linalg.norm(sph3_coef_baseline))
sph2_coef_reflection3 = np.zeros_like(sph1_coef)
pcm_fmm.fmm_m2m_reflection3(sph1_c-sph2_c, sph1_r, sph2_r, p, vscalesm, sph1_coef, sph2_coef_reflection3)
sph3_coef_reflection3 = np.zeros_like(sph1_coef)
pcm_fmm.fmm_m2m_reflection3(sph1_c-sph3_c, sph1_r, sph3_r, p, vscalesm, sph1_coef, sph3_coef_reflection3)
tmp2 = sph2_coef_reflection3 - sph2_coef_baseline
tmp3 = sph3_coef_reflection3 - sph3_coef_baseline
print("fmm_m2m_reflection3:", np.linalg.norm(tmp2) / np.linalg.norm(sph2_coef_baseline),
      np.linalg.norm(tmp3) / np.linalg.norm(sph3_coef_baseline))
sph2_coef_fast = np.zeros_like(sph1_coef)
pcm_fmm.fmm_m2m_fast(sph1_c-sph2_c, sph1_r, sph2_r, p, vscalesm, sph1_coef, sph2_coef_fast)
sph3_coef_fast = np.zeros_like(sph1_coef)
pcm_fmm.fmm_m2m_fast(sph1_c-sph3_c, sph1_r, sph3_r, p, vscalesm, sph1_coef, sph3_coef_fast)
tmp2 = sph2_coef_fast - sph2_coef_baseline
tmp3 = sph3_coef_fast - sph3_coef_baseline
print("fmm_m2m_fast:", np.linalg.norm(tmp2) / np.linalg.norm(sph2_coef_baseline),
      np.linalg.norm(tmp3) / np.linalg.norm(sph3_coef_baseline))
print("Time for M2M baseline")
%timeit pcm_fmm.fmm_m2m_baseline(sph1_c-sph3_c, sph1_r, sph2_r, p, vscalesm, sph1_coef, sph2_coef_baseline)
print("Time for M2M reflection")
%timeit pcm_fmm.fmm_m2m_reflection(sph1_c-sph3_c, sph1_r, sph2_r, p, vscalesm, sph1_coef, sph2_coef_reflection)
print("Time for M2M reflection3")
%timeit pcm_fmm.fmm_m2m_reflection3(sph1_c-sph3_c, sph1_r, sph2_r, p, vscalesm, sph1_coef, sph2_coef_reflection3)
print("Time for M2M fast")
%timeit pcm_fmm.fmm_m2m_fast(sph1_c-sph3_c, sph1_r, sph3_r, p, vscalesm, sph1_coef, sph2_coef_fast)
print("Time for M2M fast (only along OZ axis)")
%timeit pcm_fmm.fmm_m2m_fast(sph1_c-sph2_c, sph1_r, sph2_r, p, vscalesm, sph1_coef, sph2_coef_fast)
print("Time for M2M ztranslate")
%timeit pcm_fmm.fmm_m2m_ztranslate(1, sph1_r, sph3_r, p, vscalesm, sph1_coef, sph2_coef_fast)

fmm_m2m_reflection: 2.722502933198414e-16 3.872536026777169e-15
fmm_m2m_reflection3: 2.722502933198414e-16 3.841367730193296e-15
fmm_m2m_fast: 2.722502933198414e-16 1.519058154579394e-15
Time for M2M baseline
176 µs ± 577 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Time for M2M reflection
100 µs ± 390 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Time for M2M reflection3
58.2 µs ± 1.89 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Time for M2M fast
48.1 µs ± 560 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Time for M2M fast (only along OZ axis)
10.5 µs ± 60.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Time for M2M ztranslate
9.54 µs ± 44.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [4]:
# Check rotation-based M2L
import numpy as np
from pcm_fmm import pcm_fmm
pm = 20
pl = 20
ngrid = 6
vscales, w, grid, vgrid = pcm_fmm.init_globals(pm, pl, ngrid)
sph1_c = np.zeros(3)
sph1_r = 1
sph1_coef = np.ones((pm+1)*(pm+1), dtype=np.float64)
sph2_c = np.array([0, 0, -10], dtype=np.float64)
sph2_r = sph1_r + np.linalg.norm(sph2_c-sph1_c)
sph3_c = np.array([1, 2, 10], dtype=np.float64)
sph3_r = sph1_r + np.linalg.norm(sph3_c-sph1_c)
sph2_coef_baseline = np.zeros((pl+1)*(pl+1), dtype=np.float64)
pcm_fmm.fmm_m2l_baseline(sph1_c-sph2_c, sph1_r, sph2_r, pm, pl, vscales, sph1_coef, sph2_coef_baseline)
sph3_coef_baseline = np.zeros_like(sph2_coef_baseline)
pcm_fmm.fmm_m2l_baseline(sph1_c-sph3_c, sph1_r, sph3_r, pm, pl, vscales, sph1_coef, sph3_coef_baseline)
sph2_coef_fast = np.zeros_like(sph2_coef_baseline)
pcm_fmm.fmm_m2l_fast(sph1_c-sph2_c, sph1_r, sph2_r, pm, pl, vscales, sph1_coef, sph2_coef_fast)
sph3_coef_fast = np.zeros_like(sph2_coef_baseline)
pcm_fmm.fmm_m2l_fast(sph1_c-sph3_c, sph1_r, sph3_r, pm, pl, vscales, sph1_coef, sph3_coef_fast)
tmp2 = sph2_coef_fast - sph2_coef_baseline
tmp3 = sph3_coef_fast - sph3_coef_baseline
print("fmm_m2l_fast:", np.linalg.norm(tmp2) / np.linalg.norm(sph2_coef_baseline),
      np.linalg.norm(tmp3) / np.linalg.norm(sph3_coef_baseline))
print("Time for M2L baseline")
%timeit pcm_fmm.fmm_m2l_baseline(sph1_c-sph3_c, sph1_r, sph3_r, pm, pl, vscales, sph1_coef, sph3_coef_baseline)
print("Time for M2L fast")
%timeit pcm_fmm.fmm_m2l_fast(sph1_c-sph3_c, sph1_r, sph3_r, pm, pl, vscales, sph1_coef, sph3_coef_fast)
print("Time for M2L fast (only along OZ axis)")
%timeit pcm_fmm.fmm_m2l_fast(sph1_c-sph2_c, sph1_r, sph2_r, pm, pl, vscales, sph1_coef, sph2_coef_fast)
print("Time for M2L ztranslate")
%timeit pcm_fmm.fmm_m2l_ztranslate(1, sph1_r, sph2_r, pm, pl, vscales, sph1_coef, sph2_coef_fast)

fmm_m2l_fast: 6.098712141418411e-16 2.2722538450062307e-15
Time for M2L baseline
736 µs ± 4.41 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Time for M2L fast
52.5 µs ± 915 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Time for M2L fast (only along OZ axis)
14.2 µs ± 41.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Time for M2L ztranslate
13.4 µs ± 118 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [5]:
# Check rotation-based L2L
import numpy as np
from pcm_fmm import pcm_fmm
p = 20
ngrid = 6
vscales, w, grid, vgrid = pcm_fmm.init_globals(p, p, ngrid)
vscalesl = vscales[:(p+1)*(p+1)]
sph1_c = np.zeros(3)
sph1_r = 1
sph1_coef = np.ones((p+1)*(p+1), dtype=np.float64)
sph2_c = np.array([0, 0, 10], dtype=np.float64)
sph2_r = sph1_r + np.linalg.norm(sph2_c-sph1_c)
sph3_c = np.array([1, 2, 10], dtype=np.float64)
sph3_r = sph1_r + np.linalg.norm(sph3_c-sph1_c)
sph2_coef_baseline = np.zeros((p+1)*(p+1), dtype=np.float64)
pcm_fmm.fmm_l2l_baseline(sph1_c-sph2_c, sph1_r, sph2_r, p, vscalesl, sph1_coef, sph2_coef_baseline)
sph3_coef_baseline = np.zeros_like(sph2_coef_baseline)
pcm_fmm.fmm_l2l_baseline(sph1_c-sph3_c, sph1_r, sph3_r, p, vscalesl, sph1_coef, sph3_coef_baseline)
sph2_coef_fast = np.zeros_like(sph2_coef_baseline)
pcm_fmm.fmm_l2l_fast(sph1_c-sph2_c, sph1_r, sph2_r, p, vscalesl, sph1_coef, sph2_coef_fast)
sph3_coef_fast = np.zeros_like(sph2_coef_baseline)
pcm_fmm.fmm_l2l_fast(sph1_c-sph3_c, sph1_r, sph3_r, p, vscalesl, sph1_coef, sph3_coef_fast)
tmp2 = sph2_coef_fast - sph2_coef_baseline
tmp3 = sph3_coef_fast - sph3_coef_baseline
print("fmm_l2l_fast:", np.linalg.norm(tmp2) / np.linalg.norm(sph2_coef_baseline),
      np.linalg.norm(tmp3) / np.linalg.norm(sph3_coef_baseline))
print("Time for L2L baseline")
%timeit pcm_fmm.fmm_l2l_baseline(sph1_c-sph3_c, sph1_r, sph3_r, p, vscalesl, sph1_coef, sph3_coef_baseline)
print("Time for L2L fast")
%timeit pcm_fmm.fmm_l2l_fast(sph1_c-sph3_c, sph1_r, sph3_r, p, vscalesl, sph1_coef, sph3_coef_fast)
print("Time for L2L fast (only along OZ axis)")
%timeit pcm_fmm.fmm_l2l_fast(sph1_c-sph2_c, sph1_r, sph2_r, p, vscalesl, sph1_coef, sph2_coef_fast)
print("Time for L2L ztranslate")
%timeit pcm_fmm.fmm_l2l_ztranslate(1, sph1_r, sph2_r, p, vscalesl, sph1_coef, sph2_coef_fast)

fmm_l2l_fast: 1.9197264029805683e-16 8.636635476180166e-16
Time for L2L baseline
141 µs ± 588 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Time for L2L fast
46.5 µs ± 848 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Time for L2L fast (only along OZ axis)
8.67 µs ± 19.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Time for L2L ztranslate
7.8 µs ± 61.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [1]:
import numpy as np
from time import time
from pcm_fmm import pcm_fmm
from mydx import mydx
mydx.init()
nbasis, ngrid, nsph = mydx.get_sizes()
csph, rsph = mydx.get_spheres(nsph)
ui = mydx.get_ui(nsph, ngrid)
p = int(nbasis**0.5)-1
print("nsph = {}\np = {}\nngrid = {}".format(nsph, p, ngrid))
vscales, w, grid, vgrid = pcm_fmm.init_globals(p, p, ngrid)
ind = np.arange(1, nsph+1, dtype=np.int32)
cluster, children, parent, cnode, rnode, snode = pcm_fmm.btree1_init(csph, rsph, ind)
height = pcm_fmm.tree_get_height(parent)
n2 = pcm_fmm.tree_improve_get_size(children, height)
#print(height, n2)
cluster2, children2, parent2, cnode2, rnode2 = pcm_fmm.tree_improve(n2, height, cluster, children,
                                                                    parent, cnode, rnode)
#print(children.shape, children2.shape)
#print(ind)
#print(cluster)
#print(children)
np.random.seed(0)
coef_sph = np.random.randn(nbasis, nsph)

nsph = 1413
p = 6
ngrid = 110


In [2]:
t0 = time()
coef_ddpcm = mydx.ddpcm_dx(coef_sph)
print("ddPCM: ", time()-t0, " seconds")

ddPCM:  22.449885845184326  seconds


In [5]:
t0 = time()
coef_out = pcm_fmm.pcm_matvec_grid_treecode(csph, rsph, grid, w, vgrid, ui, p, vscales[:(p+1)*(p+1)], ind, \
                                   cluster, children, cnode, rnode, snode, coef_sph)
print("TREECODE: ", time()-t0, " seconds")
print(np.linalg.norm(coef_ddpcm-coef_out), np.linalg.norm(coef_ddpcm))

TREECODE:  11.970205068588257  seconds
0.05281673994514162 541.4104754512564


In [6]:
t0 = time()
coef_out2 = pcm_fmm.pcm_matvec_grid_treecode2(csph, rsph, grid, w, vgrid, ui, p, p, vscales, ind, \
                                   cluster, children, cnode, rnode, coef_sph)
print("TREECODE2: ", time()-t0, " seconds")
print(np.linalg.norm(coef_ddpcm-coef_out2), np.linalg.norm(coef_ddpcm))

TREECODE2:  4.820950031280518  seconds
0.16609022243495578 541.4104754512564


In [3]:
lwork = nsph*400
iwork = np.zeros(1, dtype=np.int32)
jwork = np.zeros(1, dtype=np.int32)
work = np.zeros((3,lwork), dtype=np.int32, order='F')
nnfar, nfar, nnnear, nnear = pcm_fmm.tree_get_farnear_work(children, cnode, rnode, iwork, jwork, work)
print(nnfar, nnnear)
if(iwork[0] != jwork[0]+1):
    print("LWORK IS TOO SMALL")
else:
    sfar, far, snear, near = pcm_fmm.tree_get_farnear(jwork[0], work, nnfar, nfar, nnnear, nnear)
    #print(nnnear, nnear, snear, near)
    #print(nnfar, nfar, sfar, far)
    t0 = time()
    coef_out_fmm = pcm_fmm.pcm_matvec_grid_fmm_baseline(csph, rsph, grid, w, vgrid, ui, p, p, vscales,
                                                        ind, cluster, children, cnode, rnode, sfar, far,
                                                        snear, near, coef_sph)
    print("FMM baseline: ", time()-t0, " seconds",
          np.linalg.norm(coef_ddpcm-coef_out_fmm)/np.linalg.norm(coef_ddpcm))
    t0 = time()
    coef_out_fmm = pcm_fmm.pcm_matvec_grid_fmm_fast(csph, rsph, grid, w, vgrid, ui, p, p, vscales,
                                                        ind, cluster, children, cnode, rnode, sfar, far,
                                                        snear, near, coef_sph)
    print("FMM fast: ", time()-t0, " seconds",
          np.linalg.norm(coef_ddpcm-coef_out_fmm)/np.linalg.norm(coef_ddpcm))
    #print(coef_out_fmm)
    #print(coef_out_fmm-coef_out)
    #print(coef_out)

162326 45299
FMM baseline:  5.728753089904785  seconds 0.0004234202990096702
FMM fast:  3.465308904647827  seconds 0.0004234202990096688


In [4]:
lwork = nsph*400
iwork = np.zeros(1, dtype=np.int32)
jwork = np.zeros(1, dtype=np.int32)
work = np.zeros((3,lwork), dtype=np.int32, order='F')
nnfar, nfar, nnnear, nnear = pcm_fmm.tree_get_farnear_work(children2, cnode2, rnode2, iwork, jwork, work)
if(iwork[0] != jwork[0]+1):
    print("LWORK IS TOO SMALL")
else:
    sfar, far, snear, near = pcm_fmm.tree_get_farnear(jwork[0], work, nnfar, nfar, nnnear, nnear)
    #print(nnnear, nnear, snear, near)
    #print(nnfar, nfar, sfar, far)
    t0 = time()
    coef_out_fmm = pcm_fmm.pcm_matvec_grid_fmm_fast(csph, rsph, grid, w, vgrid, ui, p, p, vscales,
                                                        ind, cluster2, children2, cnode2, rnode2, sfar, far,
                                                        snear, near, coef_sph)
    print("FMM fast (improved tree): ", time()-t0, " seconds",
          np.linalg.norm(coef_ddpcm-coef_out_fmm)/np.linalg.norm(coef_ddpcm))
    #print(coef_out_fmm)
    #print(coef_out_fmm-coef_out)
    #print(coef_out)

FMM fast (improved tree):  1.7502851486206055  seconds 0.9685380091602923
